In [13]:
import numpy as np
import tensorflow
import keras
import json
import os
import re
from PIL import Image
from tqdm.notebook import tqdm

In [14]:
# load list of labels from json
with open('dataset\\labels.json', 'r') as f:
    labels_loaded = json.load(f)

# load list of correct matches from json
with open('dataset\\matched_samples.json', 'r') as f:
    matched_samples = json.load(f)

In [9]:
print(len(labels_loaded))
print(len(matched_samples))

48040
41495


In [16]:
"""    LABELS CHART
0	C3
1	D3
2	E3
3	F3
4	G3
5	A3
6	B3
7	C4
8	D4
9	E4
10	F4
11	G4
12	A4
13	B4
14	C5
15	D5
16	E5
17	F5
18	G5
19	A5
20	B5
21	C6
22	D6
23	E6
24	F6
25	G6
26	A6
27	B6
28	rest
29  double_whole
30	whole
31	half
32	quarter
33	eighth
34	sixteenth
35	thirty_second
36  sixty_fourth
37  hundred_twenty_eighth
38	dotted
"""
    
notes = ['C', 'D', 'E', 'F', 'G', 'A', 'B']
durations = ['double_whole', 'whole', 'half', 'quarter', 'eighth', 'sixteenth', 'thirty_second', 'sixty_fourth', 'hundred_twenty_eighth']
label_arrays = []
    
for i in tqdm(matched_samples[:1]):
    labels = labels_loaded[i]
    for label in labels:
        print(label)
        # ignore fermatas
        label = label.replace("_fermata", "")
        label_array = np.zeros((39))
        
        note = label[5]
        octave = int(label[6])
        # in case of dotted or double-dotted
        if '..' in label:
            duration = label[8:len(label) - 2]
        elif '.' in label:
            duration = label[8:len(label) - 1]
        else:
            duration = label[8:]
        
        duration_index = durations.index(duration)
        duration_label = 29 + duration_index
        label_array[duration_label] = 1
        print(f"duration_label={duration_label}")
        
        note_index = notes.index(note)
        note_label = note_index + 7 * (octave - 3)
        label_array[note_label] = 1
        print(f"note_label={note_label}")
        
        if '.' in label:
            label_array[38] = 1
        #label_arrays.append(label_array)

#label_arrays = np.array(label_arrays)
#print(label_arrays.shape)

  0%|          | 0/1 [00:00<?, ?it/s]

note-B5_quarter
duration_label=32
note_label=20
note-E5_eighth
duration_label=33
note_label=16
note-B5_eighth
duration_label=33
note_label=20
note-C6_eighth
duration_label=33
note_label=21
note-B5_eighth
duration_label=33
note_label=20
note-A5_eighth
duration_label=33
note_label=19
note-A5_eighth
duration_label=33
note_label=19
note-A5_sixteenth
duration_label=34
note_label=19
note-G5_sixteenth
duration_label=34
note_label=18
note-A5_sixteenth
duration_label=34
note_label=19
note-B5_sixteenth
duration_label=34
note_label=20
note-A5_sixteenth
duration_label=34
note_label=19
note-G5_sixteenth
duration_label=34
note_label=18
note-A5_sixteenth
duration_label=34
note_label=19


In [3]:
np.save('dataset\\labels_as_arrays', label_arrays)

In [2]:
label_arrays = np.load('dataset\\labels_as_arrays.npy')

In [66]:
x_train = []

for file in os.listdir("D:\\primus_notes"):
    img = Image.open(f"D:\\primus_notes\\{file}").convert("L")
    img.load()
    data = np.asarray(img, dtype='uint8')
    x_train.append(data)

  0%|          | 0/684784 [00:00<?, ?it/s]

In [67]:
x_train = np.array(x_train)
print(x_train.shape)

(684784, 149, 40)


In [7]:
x_train = x_train.reshape(684784, 40, 149, 1)

In [90]:

np.save('D:\\data_as_numpy_array', x_train)

In [7]:
w = 40
h = 149
n = 684784

x_train = np.load('D:\\data_as_numpy_array.npy')
y_train = np.load('dataset\\labels_as_arrays.npy')
x_train = x_train.reshape((n, w, h, 1))

In [11]:
print(y_train[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [ ]:
x_train = x_train.astype('float16')
x_train = x_train/255

In [14]:
print(x_train[0].shape)

(40, 149, 1)


In [3]:
print(x_train.shape)
print(y_train.shape)

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, (30, 30), activation=keras.activations.relu, input_shape=(40, 149, 1)))
model.add(keras.layers.Conv2D(32, (10, 10), activation=keras.activations.relu))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(w * h, activation=keras.activations.relu))
model.add(keras.layers.Dense(39, activation=keras.activations.softmax))

model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

(684784, 40, 149, 1)
(684784, 39)


c:\Users\Seyren\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 38, 147, 10)    │           100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 19, 73, 10)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 17, 71, 10)     │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 35, 10)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2800)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5960)           │    16,693,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 39)             │       232,479 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,927,449 (64.57 MB)

 Trainable params: 16,927,449 (64.57 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/OMR/CNN_ONLY.keras",
                                                save_best_only=True,
                                                monitor='val_accuracy',
                                                mode='max')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True,
                                                  monitor='val_accuracy',
                                                  mode='max')

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=200,
                    validation_split=0.2,
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/3
 182/4280 ━━━━━━━━━━━━━━━━━━━━ 16:35 243ms/step - accuracy: 0.0358 - loss: 12270743.0000

KeyboardInterrupt: 